# Auto post to Social site
Schedule: 1 Image post per day per Page (setup n cron items, each item has 1 category in param)

In [1]:
from PIL import Image, ImageDraw, ImageFont
import pymongo
import os
import sys
import time

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [3]:
def get_timestamp():
    return str(int(time.time()))

In [4]:
CURRENT_PATH = '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/'

In [5]:
def wrap_text(text, font, max_width, draw):
    """
    Splits text into lines that fit within max_width
    """
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + (" " if current_line else "") + word
        w = draw.textlength(test_line, font=font)

        if w <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    return lines

In [6]:
def draw_wrapped_text(draw, position, text, font, max_width, fill=(255,255,255), line_spacing=6):
    x, y = position
    lines = wrap_text(text, font, max_width, draw)

    for line in lines:
        draw.text((x, y), line, font=font, fill=fill)
        y += font.size + line_spacing


In [36]:
#2 or 3 certifications in 1 image
LOGO_POSITIONS = [(70, 290), (70, 630), (70, 985)]
NAME_POSITIONS = [(410, 350), (410, 695), (410, 1035)]  #certs name
SALARY_POSITION = (200, 130)    #salary range
FOOTER_URL_POSITION = (350, 1300)   #https://udemy/...

IMG_TEMPLATE_PATH = CURRENT_PATH + 'template/img_template.png'
IMG_LOGO_PATH_PREFIX = CURRENT_PATH + 'logo/'

IMG_OUTPUT_PATH_PREFIX = CURRENT_PATH + 'output/'  #the final image
FONT_PATH = CURRENT_PATH + "font/Swansea-q3pd.ttf"

In [8]:
#define category and link to show footnote in image or video
CATEGORY_LINK_MAP = {
    'GENERAL': 'https://www.linkedin.com/company/it-certificates-pool',
    'AI': 'https://www.linkedin.com/company/it-certificates-pool'
}

In [9]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']   
tb_advertising_posts = db['tb_advertising_posts']   #store contents to post in social media
tb_cert_metadata = db['tb_cert_metadata']    #meta data of certificates

In [10]:
def find_1_content_from_db(_category):
    return tb_advertising_posts.find_one({'posted_img': 0, 'category': _category})  #not yet post any image

In [34]:
def create_certifications_image(
    _category:str,
    cert_symbols: list,
    cert_names: list,
    salary_range: str,
    output_path: str
):
    base = Image.open(IMG_TEMPLATE_PATH).convert("RGBA")
    draw = ImageDraw.Draw(base)
    index = 0
    # ---- Add logo images ----
    for symbol in cert_symbols:
        overlay_img = Image.open(IMG_LOGO_PATH_PREFIX + symbol + '.png').convert("RGBA")
        #don't need to resize because logos are saved with size 330x330
        # if "size" in item and item["size"]:
        #     overlay_img = overlay_img.resize(item["size"], Image.ANTIALIAS)
        base.paste(overlay_img, LOGO_POSITIONS[index], overlay_img)
        index = index + 1

    # ---- Add certification names ----
    index = 0
    for t in cert_names:
        font = ImageFont.truetype(FONT_PATH, 48)
        draw_wrapped_text(
            draw=draw,
            position= NAME_POSITIONS[index],
            text= t,
            font= font,
            max_width= 620,
            fill= (0, 0, 153),   #certification name color
            line_spacing=8
        )
        index = index + 1
    #draw salary range
    draw_wrapped_text(
            draw=draw,
            position= SALARY_POSITION,
            text= 'Salary range: ' + salary_range,
            font= ImageFont.truetype(FONT_PATH, 60),
            max_width= 800,
            fill= (153, 153, 0)   #text color
        )
    #draw footer
    draw_wrapped_text(
            draw=draw,
            position= FOOTER_URL_POSITION,
            text= CATEGORY_LINK_MAP[_category],
            font= ImageFont.truetype(FONT_PATH, 20),
            max_width= 620,
            fill= (0, 0, 255)
        )

    base.convert("RGB").save(output_path, "PNG")
    print(f"Final image saved to: {output_path}")

#test
# create_image_with_text_and_overlays('./template/img_template_3_certs.png', overlays, texts, './output/3_certs.png')

In [ ]:
def post_1_image_by_category(_category):
    post_details = find_1_content_from_db(_category)
    if post_details is None:
        print('There is no post for Image content of category: ' + _category)
        return
    if 'symbols' not in post_details:
        print('There is no symbol for Image content of category: ' + _category)
        return
    symbols = post_details['symbols'].split(',')
    #find name of certifications
    cert_names = []
    for symbol in symbols:
        cert_details = tb_cert_metadata.find_one({'symbol':symbol})
        if cert_details is None:
            print('There is no detail for the certification: ' + symbol)
            return
        cert_names.append(cert_details['name'])
    #create final image
    new_name = IMG_OUTPUT_PATH_PREFIX + post_details['symbols'].replace(',', '-') + '.png'
    create_certifications_image(_category, symbols, cert_names, post_details['salary'], new_name)
    #post to social network

In [35]:
post_1_image_by_category('AI')

Final image saved to: /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/output/PMI_CPMAI-GCP_GAL.png


In [ ]:
# if __name__ == '__main__':
#     args = sys.argv
#     print(args)
#     if len(args) > 1:
#         category = args[1]   #category of certifications
#         print(category)
#         post_1_image_by_category(category)

['/Users/sang/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json']
--f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json
There is no post for Image content of category: --f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json
